In [2]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

first_year = 1985
last_year = 2020

data_folder = "projects/amazon-forest-regrowth/assets"




Output()

In [2]:
# This collection is not publicly accessible. To sign up for access,
# please see https://developers.planet.com/docs/integrations/gee/nicfi
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')

# # Filter basemaps by date and get the first image from filtered results
# basemap = nicfi.filter(ee.Filter.date('2016-03-01', '2016-07-01')).first()
# basemap2 = nicfi.filter(ee.Filter.date('2018-03-01', '2018-07-01')).first()
basemap = nicfi.filter(ee.Filter.date('2023-01-01', '2023-07-01')).first()

vis_planet = {'bands': ['R', 'G', 'B'], 'min': 64, 'max': 5454, 'gamma': 1.8}

age = ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_secondary_vegetation_age_v1")




In [19]:
import ee
import geemap

# Initialize the Earth Engine module
ee.Initialize()

# Define the NICFI Image Collection
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')

# Define visualization parameters for Planet basemaps
vis_planet = {'bands': ['R', 'G', 'B'], 'min': 64, 'max': 5454, 'gamma': 1.8}

all_ages = ee.Image(f"projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_secondary_vegetation_age_v1")

# Define a function to get secondary vegetation age for a specific year
def get_age_image(year):
    return all_ages.select(f"secondary_vegetation_age_{year}")

# Create an ImageCollection for each year from 2020 to 2023
years = [2020, 2021, 2022, 2023]
age_images = ee.ImageCollection([get_age_image(year) for year in years])

for year in years:
    age_image = get_age_image(year)
    print(f"Year: {year}, Bands: {age_image.bandNames().getInfo()}")

map = geemap.Map()
map.addLayer(all_ages.select("secondary_vegetation_age_1986"), {'min': 0, 'max': 30, 'palette': ['red', 'green', 'blue']}, '1985')
map.addLayer(age_images.first(), {'min': 0, 'max': 30, 'palette': ['red', 'green', 'blue']}, '2020')
map


Year: 2020, Bands: ['secondary_vegetation_age_2020']
Year: 2021, Bands: ['secondary_vegetation_age_2021']
Year: 2022, Bands: ['secondary_vegetation_age_2022']
Year: 2023, Bands: ['secondary_vegetation_age_2023']


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [12]:

# Create a GIF from the ImageCollection
gif_params = {
    'dimensions': 600,
    'region': ee.Geometry.Rectangle([-75, -33, -34, 5]), # Adjust this region as needed
    'framesPerSecond': 2,
    'crs': 'EPSG:3857'
}

# Export the GIF to Google Drive
task = ee.batch.Export.video.toDrive(
    collection=age_images,
    description='Secondary_Forest_Age_GIF',
    **gif_params
)

task.start()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'secondary_vegetation_age_2020',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': 0,
      'max': 255},
     'dimensions': [155239, 158459],
     'crs': 'EPSG:4326',
     'crs_transform': [0.00026949458523585647,
      0,
      -74.89739613957876,
      0,
      -0.00026949458523585647,
      7.91209152793951]}],
   'version': 1724225835531157,
   'id': 'projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_secondary_vegetation_age_v1',
   'properties': {'system:footprint': {'type': 'LinearRing',
     'coordinates': [[-44.99120091146869, -34.79188619133721],
      [-39.925146999777766, -34.79188617797568],
      [-33.06113219577002, -34.791885974087286],
      [-33.06118844262496, 7.9122266386845],
      [-40.25198918862905, 7.912226776789873],
      [-45.154622064354704, 7.912226754546059],
      [-54.30620338690848, 7.912226723010938],


In [6]:
roi = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filterMetadata("country_na", "equals", "Panama").geometry()

# Load the image collections
transition = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes').mosaic().clip(roi)
annual_changes = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(roi)

# Define regrowth and degraded conditions
regrowth = transition.gte(31).And(transition.lte(33))

# Initialize AgeRegrowth and AgeDegraded
age = ee.Image.constant(0)

# Calculate AgeRegrowth
for i in range(1990, 2023):
    year = 'Dec' + str(i)
    annual_changes_year = annual_changes.select(year)
    condition = annual_changes_year.eq(4).And(regrowth) # were regrowing then AND are regrowing now
    age = age.add(condition.eq(1))

age = age.selfMask().rename("age")

# Visualization parameters
vis_ages = {'min': 0, 'max': 35, 'palette': ['yellow', 'red']}

Map = geemap.Map()
Map.centerObject(basemap, 4)
Map.addLayer(basemap, vis_planet, '2023 mosaic')
Map.addLayer(age, vis_ages, 'ages')
Map

Map(center=[-0.0013335310118573354, -73.54611700899127], controls=(WidgetControl(options=['position', 'transpa…

In [3]:
mature = ee.Image(f"{data_folder}/mature_biomass")
agbd = ee.Image(f"{data_folder}/age_agbd").select('agbd')
mask = ee.Image(f"{data_folder}/mapbiomas/one_hectare_mask")
ages = ee.Image(f"{data_folder}/mapbiomas/secondary").select('age').updateMask(mask)
pantanal = ee.FeatureCollection(f"{data_folder}/raw/biomes_br") \
        .filter(ee.Filter.eq("CD_Bioma", 6)).geometry()

# Visualization parameters
vis_ages = {'min': 0, 'max': 35, 'palette': ['yellow', 'red']}
vis_agbd = {'min': 0, 'max': 300, 'palette': ['yellow', 'green']}
vis_mature = {'min': 0, 'max': 300, 'palette': ['blue', 'red']}
vis_geom = {
    'color': 'red',      # Color of the edge
    'width': 2           # Width of the edge
}

Map = geemap.Map()
Map.centerObject(basemap, 4)
# Map.addLayer(basemap, vis, '2016 mosaic')
Map.addLayer(basemap, vis_planet, '2020 mosaic')
# Map.addLayer(pantanal, vis_geom, 'pantanal')
Map.addLayer(ages, vis_ages, 'ages')
# Map.addLayer(lulc, {}, 'lulc')
# Map.addLayer(agbd, vis_agbd, 'agbd')
# Map.addLayer(mature, vis_mature, 'mature')
Map

Map(center=[-0.03247460684437231, -73.82115720575285], controls=(WidgetControl(options=['position', 'transpare…